In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.optimize import least_squares
import math
import pandas as pd
import csv
from scipy import stats
from scipy import optimize
from datetime import date, timedelta
import random
import requests
from scipy.stats import truncnorm

# Data description

In [2]:
df = pd.read_csv('/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/data/covid.csv')

In [23]:
pop = pd.read_csv('/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/results/seiir_bootstrap/populacao_municipio.csv')

In [3]:
df.head()

,epidemiological_week,date,order_for_place,state,city,city_ibge_code,place_type,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,new_confirmed,last_available_deaths,new_deaths,last_available_death_rate,estimated_population_2019,is_last,is_repeated
0,23,2020-06-01,65,AC,Acrelândia,1200013.0,city,158,1035.65810,0,1,0,0.0063,15256.0,True,False
1,23,2020-06-01,32,AC,Assis Brasil,1200054.0,city,28,377.51112,1,2,0,0.0714,7417.0,True,False
2,23,2020-06-01,24,AC,Brasiléia,1200104.0,city,38,144.60766,1,2,0,0.0526,26278.0,True,False
3,23,2020-06-01,55,AC,Bujari,1200138.0,city,49,477.30372,0,2,0,0.0408,10266.0,True,False
4,23,2020-06-01,25,AC,Capixaba,1200179.0,city,40,340.91878,0,1,0,0.0250,11733.0,True,False


In [14]:
df = df[df.date <= '2020-05-22']

In [15]:
df.state.unique()

array(['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG',
       'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR',
       'RS', 'SC', 'SE', 'SP', 'TO'], dtype=object)

In [16]:
# Number of confirmed cases up to May 22

df.new_confirmed.sum()

334555

In [31]:
76871/334555

0.22977088968928874

In [22]:
# Number of confirmed cases up to May 22 by states

df.groupby(['state'])['new_confirmed'].sum()

state
AC     3343
AL     5300
AM    27038
AP     5655
BA    12558
CE    34573
DF     5948
ES     9520
GO     2344
MA    18767
MG     5995
MS      805
MT     1271
PA    22697
PB     6882
PE    25760
PI     3258
PR     2985
RJ    33589
RN     4598
RO     2774
RR     2296
RS     5918
SC     6458
SE     4922
SP    76871
TO     2430
Name: new_confirmed, dtype: int64

In [27]:
pop.groupby(['ARmaior'])['Total'].sum()

ARmaior
ACRE                   8.944700e+05
ALAGOAS                3.351092e+06
AMAPÁ                  8.617730e+05
AMAZONAS               4.207714e+06
BAHIA                  1.493042e+07
CEARÁ                  9.187886e+06
DISTRITO FEDERAL       3.052546e+06
ESPÍRITO SANTO         4.064052e+06
GOIÁS                  7.116143e+06
MARANHÃO               7.114598e+06
MATO GROSSO            3.526220e+06
MATO GROSSO DO SUL     2.809394e+06
MINAS GERAIS           2.129267e+07
PARANÁ                 1.151684e+07
PARAÍBA                4.039277e+06
PARÁ                   8.690745e+06
PERNAMBUCO             9.617072e+06
PIAUÍ                  3.280697e+06
RIO DE JANEIRO         1.736619e+07
RIO GRANDE DO NORTE    3.534165e+06
RIO GRANDE DO SUL      1.142297e+07
RONDÔNIA               1.796460e+06
RORAIMA                6.311810e+05
SANTA CATARINA         7.252502e+06
SERGIPE                2.319032e+06
SÃO PAULO              4.628933e+07
TOCANTINS              1.590248e+06
Name: Total, dtype: 

AC, Acre; AL, Alagoas; AP, Amapá; AM, Amazonas; BA, Bahia; CE, Ceará; 

DF, Distrito Federal; ES, Espírito Santo; GO, Goiás; MA, Maranhão; MT, Mato Grosso; 

MS, Mato Grosso do Sul; MG, Minas Gerais; PA, Pará; PB, Paraíba; PR, Paraná; PE, Pernambuco;

PI, Piauí; RJ, Rio de Janeiro; RN, Rio Grande do Norte; RS, Rio Grande do Sul; 

RO, Rondônia; RR, Roraima; SC, Santa Catarina; SP, São Paulo; SE, Sergipe; TO, Tocantins.

In [30]:
print('AC',   3343*100000/8.944700e+05)
print('AL',     5300*100000/3.351092e+06)
print('AM',    27038*100000/4.207714e+06)
print('AP',    5655*100000/8.617730e+05)
print('BA',    12558*100000/1.493042e+07)
print('CE',    34573*100000/9.187886e+06)
print('DF',     5948*100000/3.052546e+06)
print('ES',     9520*100000/4.064052e+06)
print('GO',    2344*100000/7.116143e+06)
print('MA',    18767*100000/7.114598e+06)
print('MG',     5995*100000/2.129267e+07)
print('MS',      805*100000/2.809394e+06)
print('MT',     1271*100000/3.526220e+06)
print('PA',    22697*100000/8.690745e+06)
print('PB',     6882*100000/4.039277e+06)
print('PE',    25760*100000/9.617072e+06)
print('PI',     3258*100000/3.280697e+06)
print('PR',     2985*100000/1.151684e+07)
print('RJ',    33589*100000/1.736619e+07)
print('RN',     4598*100000/3.534165e+06)
print('RO',     2774*100000/1.796460e+06)
print('RR',     2296*100000/6.311810e+05)
print('RS',     5918*100000/1.142297e+07)
print('SC',     6458*100000/7.252502e+06)
print('SE',     4922*100000/2.319032e+06)
print('SP',    76871*100000/4.628933e+07)
print('TO',     2430*100000/1.590248e+06)           

AC 373.7408744843315
AL 158.15740063239087
AM 642.5816963795543
AP 656.2052883996133
BA 84.11015899083884
CE 376.28895264917304
DF 194.853738485841
ES 234.2489712237934
GO 32.939191918993195
MA 263.7815938440935
MG 28.15522900603823
MS 28.653866278635178
MT 36.044262694897085
PA 261.1628807426751
PB 170.3770253934058
PE 267.8569943117822
PI 99.30816530755507
PR 25.91856794051146
RJ 193.4160572929353
RN 130.10145253546452
RO 154.4147935384033
RR 363.7625340433251
RS 51.80789234323473
SC 89.04513228676117
SE 212.24372928014793
SP 166.06634833556674
TO 152.8063547320921


In [18]:
# Number of deaths up to May 22

df.new_deaths.sum()

21146

In [17]:
# Number of confirmed cases up to March 25

df[df.date <= '2020-03-25'].new_confirmed.sum()

2566

In [19]:
# Number of deaths up to March 25

df[df.date <= '2020-03-25'].new_deaths.sum()

59

# Analysis of the fit results

In [2]:
df = pd.read_csv('/media/juliane_oliveira/My Passport/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/First review/Data and code analisys/outputs_testing/test_seiir_fits_boots.csv')

In [3]:
df.head()

,Unnamed: 0,state,type,beta0_ls,beta0_ml,beta1_ls,beta1_ml,tcut_ls,tcut_ml,delta_ls,delta_ml,e0_ls,e0_ml,ia0_ls,ia0_ml,is0_ls,is0_ml
0,0,AM,estado,1.233434,1.763611,0.591353,0.708238,93.952321,94.368859,0.700000,0.515792,0.000002,4.753044e-07,0.000002,2.592675e-07,5.443991e-08,6.869455e-07
1,1,AM,estado,2.000000,1.549807,1.098894,0.748755,96.350096,98.546809,0.247201,0.460819,0.000002,1.031083e-07,0.000002,5.626661e-07,1.172151e-06,1.379001e-06
2,2,AM,estado,1.155266,1.515730,0.597303,0.812038,94.766717,98.251489,0.699067,0.411809,0.000002,1.006499e-06,0.000002,1.721702e-06,5.789328e-07,1.715864e-06
3,3,AM,estado,1.997959,1.634999,1.019994,0.872344,93.934599,95.089442,0.290449,0.373129,0.000002,1.522335e-06,0.000002,1.858073e-06,8.602349e-07,1.931153e-06
4,4,AM,estado,1.213843,1.943396,0.593559,0.846689,94.450127,90.254304,0.700000,0.389947,0.000002,1.428494e-06,0.000002,1.129438e-06,5.052882e-08,1.651239e-06


In [11]:
import scipy.stats


def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.median(a), scipy.stats.sem(a)
    h = se * scipy.stats.skewnorm.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h


In [12]:
lista = [ 'beta0_ls', 'beta0_ml', 'beta1_ls','beta1_ml', 'tcut_ls', 'tcut_ml', 'delta_ls', 'delta_ml', 'e0_ls',
       'e0_ml', 'ia0_ls', 'ia0_ml', 'is0_ls', 'is0_ml']

In [13]:
for value in lista:
    print(value, mean_confidence_interval(df[value], confidence=0.95))

beta0_ls (1.9996389365139469, 1.9103379183224014, 2.0889399547054923)
beta0_ml (1.6679693834668634, 1.6276538948050636, 1.7082848721286632)
beta1_ls (1.0358874037929677, 0.9848454925618654, 1.0869293150240702)
beta1_ml (0.8440423377979086, 0.8203602607043528, 0.8677244148914645)
tcut_ls (94.90161285942946, 94.74621738941407, 95.05700832944486)
tcut_ml (97.05327512145575, 96.54751104161514, 97.55903920129637)
delta_ls (0.28229439977241605, 0.23566979240875308, 0.328919007136079)
delta_ml (0.389065808891516, 0.3688341280873538, 0.4092974896956782)
e0_ls (2.376587422016826e-06, 2.3599183740948173e-06, 2.3932564699388348e-06)
e0_ml (1.1990215284690064e-06, 1.0754532287891466e-06, 1.3225898281488663e-06)
ia0_ls (2.376587422016826e-06, 2.376587400503949e-06, 2.376587443529703e-06)
ia0_ml (1.2609030062747696e-06, 1.141286721517328e-06, 1.3805192910322113e-06)
is0_ls (8.103499569424453e-07, 7.356443539347723e-07, 8.850555599501184e-07)
is0_ml (1.4924451641082333e-06, 1.3784685651908844e-06, 1.

In [33]:
b = 1.62
delta = 0.42
p = 0.2
gammaA = 1/3.5
gammaS = 1/4

In [34]:
R0a = b*delta/gammaA

R0s = b/gammaS

R0_geral = p*R0s + (1-p)*R0a

print("Número Reprodutivo {}.".format(R0_geral))

print("Número Reprodutivo dos sintomaticos {}.".format(R0s))

print("Número Reprodutivo dos assintomáticos {}.".format(R0a))

Número Reprodutivo 3.2011200000000004.
Número Reprodutivo dos sintomaticos 6.48.
Número Reprodutivo dos assintomáticos 2.3814.
